In [ ]:
'''
Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

SELECT name
FROM country_club.Facilities
WHERE membercost != 0
'''

In [ ]:
'''
Q2: How many facilities do not charge a fee to members?
SELECT COUNT( name )
FROM country_club.Facilities
WHERE membercost = 0
'''

In [ ]:
'''
Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

SELECT facid, name, membercost, monthlymaintenance
FROM country_club.Facilities
WHERE membercost != 0
AND membercost < (monthlymaintenance*0.2)
'''

In [ ]:
'''
Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

SELECT * 
FROM country_club.Facilities
WHERE facID IN (1,5)
'''

In [ ]:
'''
Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance <=100
THEN 'cheap'
ELSE 'expensive'
END AS category
FROM country_club.Facilities
'''

In [ ]:
'''
Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

SELECT firstname, surname
FROM country_club.Members
WHERE joindate = (SELECT(max(joindate)) FROM country_club.Members)
'''

In [ ]:
'''
Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

SELECT DISTINCT CONCAT( firstname, ' ', surname ) AS fullname, f.name
FROM country_club.Members m
INNER JOIN country_club.Bookings b ON m.memid = b.memid
INNER JOIN country_club.Facilities f ON b.facid = f.facid
WHERE f.name LIKE '%Tennis Court%'
ORDER BY b.memid
'''

In [ ]:
'''
Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

SELECT f.name, CONCAT( m.firstname, ' ', m.surname ) AS FullName,
(CASE WHEN b.memid = 0 THEN guestcost ELSE f.membercost END) * b.slots AS cost
FROM country_club.Bookings b
INNER JOIN country_club.Members m ON b.memid = m.memid
INNER JOIN country_club.Facilities f ON b.facid = f.facid
WHERE LEFT (starttime, 10) = '2012-09-14'
AND ((CASE WHEN b.memid = 0 THEN f.guestcost ELSE f.membercost END) * b.slots) > 30
ORDER BY cost DESC, name
'''

In [ ]:
'''
Q9: This time, produce the same result as in Q8, but using a subquery.

SELECT facility_name, FullName, cost
FROM
    (SELECT
        (SELECT name FROM Facilities as f WHERE f.facid = b.facid) as facility_name, 
        (SELECT CONCAT( m.firstname, ' ', m.surname ) FROM Members as m WHERE m.memid = b.memid) as FullName, 
        (CASE WHEN b.memid = 0 THEN guestcost ELSE f.membercost END) * b.slots AS cost
        FROM country_club.Bookings b
        LEFT JOIN Facilities as f ON b.facid = f.facid
        WHERE LEFT (starttime, 10) = '2012-09-14'
    ) as s
WHERE cost > 30
ORDER BY cost DESC
'''

In [1]:
import sqlite3
import pandas as pd
import numpy as np

sql_connect = sqlite3.connect('pythonsqlite.db')
cursor = sql_connect.cursor()

In [2]:
# Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, /
# sorted by revenue. Remember that there's a different cost for guests and members!

low_rev = pd.read_sql_query("""
SELECT name, (membercost + guestcost + monthlymaintenance) total_revenue
FROM Facilities
WHERE membercost + guestcost + monthlymaintenance < 1000
ORDER BY total_revenue
""",sql_connect)

In [3]:
low_rev

,name,total_revenue
0,Table Tennis,15.0
1,Snooker Table,20.0
2,Pool Table,20.0
3,Badminton Court,65.5
4,Squash Court,101.0
5,Tennis Court 1,230.0
6,Tennis Court 2,230.0


In [4]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order.

recom = pd.read_sql_query("""
SELECT m1.surname || ' ' || m1.firstname AS name, m2.firstname || ' ' || m2.surname AS recommendedBy
FROM Members AS m1
INNER JOIN Members AS m2
ON m2.recommendedby = m1.memid
ORDER BY m1.surname, m1.firstname
""",sql_connect)

recom.head()

,name,recommendedBy
0,Bader Florence,Ramnaresh Sarwin
1,Baker Timothy,Joan Coplin
2,Butters Gerald,Matthew Genting
3,Farrell Jemima,Timothy Baker
4,Farrell Jemima,David Pinker


In [5]:
# Q12: Find the facilities with their usage by member, but not guests.

mem_use = pd.read_sql_query("""
SELECT memid, name, SUM(slots) AS usage
FROM Bookings 
LEFT JOIN Facilities
ON Bookings.facid = Facilities.facid
WHERE memid != 0
GROUP BY memid
""",sql_connect)

mem_use

,memid,name,usage
0,1,Table Tennis,685
1,2,Tennis Court 1,435
2,3,Massage Room 1,660
3,4,Massage Room 1,326
4,5,Tennis Court 1,409
5,6,Tennis Court 2,366
6,7,Badminton Court,267
7,8,Tennis Court 2,440
8,9,Tennis Court 2,249
9,10,Tennis Court 1,345


In [6]:
# Q13: Find the facilities usage by month, but not guests.

member_book_m = pd.read_sql_query("""
SELECT f.name AS Facility_Name,strftime("%m", starttime) as Month , SUM(slots) AS Member_Book 
FROM Bookings
INNER JOIN Facilities as f
USING (facid)
WHERE memid <> '0' 
GROUP BY f.name, strftime("%m", starttime)
""", sql_connect)

In [7]:
member_book_m

,Facility_Name,Month,Member_Book
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110
